# Deep Nets

TensorFlow has many abstractions: higher level API to simplify the code and it usually allows skipping things, at the cost of control.

Many of these abstractions reside in TensorFlow's tf.contrib section. Typically, libraries get developed and accepted into contrib and then 'graduate' to being included as part of the standard TensorFlow library. The speed of development of TensorFlow has caused abstractions to come and go quickly.

This series of lectures will focus on presenting the most common abstractions used:

- Estimator API
- Keras
- Layers

We will focus on understanding how to use these abstractions to build deep densely connected neural networks. Using these abstractions makes it easy to stack layers on top of each other for simpler tasks.

Let's start by exploring the dataset.

In [1]:
from sklearn.datasets import load_wine
wine_data = load_wine()

In [2]:
wine_data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [3]:
print(wine_data['DESCR'])

Wine Data Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- 1) Alcohol
 		- 2) Malic acid
 		- 3) Ash
		- 4) Alcalinity of ash  
 		- 5) Magnesium
		- 6) Total phenols
 		- 7) Flavanoids
 		- 8) Nonflavanoid phenols
 		- 9) Proanthocyanins
		- 10)Color intensity
 		- 11)Hue
 		- 12)OD280/OD315 of diluted wines
 		- 13)Proline
        	- class:
                - class_0
                - class_1
                - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash: 

In [4]:
feat_data = wine_data['data']

In [5]:
labels = wine_data['target']

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(feat_data, labels, test_size = 0.3, random_state=101)

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
scaler = MinMaxScaler()

scaled_x_train = scaler.fit_transform(X_train)
scaled_x_test = scaler.fit_transform(X_test)

# Estimator API

In [10]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

/home/alvaro/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
from tensorflow import estimator

In [12]:
X_train.shape

(124, 13)

In [14]:
feat_cols = [tf.feature_column.numeric_column('x', shape=[13])]

In [15]:
deep_model = estimator.DNNClassifier(hidden_units=[13,13,13],
                                    feature_columns=feat_cols,
                                    n_classes=3,
                                    optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpypkpy7i1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd04fe683c8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [16]:
input_fn = estimator.inputs.numpy_input_fn(x={'x':scaled_x_train},
                                          y=y_train,
                                          shuffle=True,
                                          batch_size=10,
                                          num_epochs=5)

In [17]:
deep_model.train(input_fn=input_fn, steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpypkpy7i1/model.ckpt.
INFO:tensorflow:loss = 10.788323, step = 1
INFO:tensorflow:Saving checkpoints for 62 into /tmp/tmpypkpy7i1/model.ckpt.
INFO:tensorflow:Loss for final step: 3.705874.


In [24]:
input_fn_eval = estimator.inputs.numpy_input_fn(x={'x':scaled_x_test},
                                          y=y_test,
                                          shuffle=False)

In [25]:
preds = list(deep_model.predict(input_fn=input_fn_eval))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpypkpy7i1/model.ckpt-62
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [26]:
predictions = [p['class_ids'][0] for p in preds]

In [27]:
from sklearn.metrics import confusion_matrix, classification_report

In [29]:
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       1.00      0.58      0.73        19
          1       0.68      0.95      0.79        22
          2       0.92      0.85      0.88        13

avg / total       0.85      0.80      0.79        54

